### report of implementations

In [143]:
from __future__ import absolute_import, division, print_function, unicode_literals
from classes2048 import state, tbl
from time import sleep
import tensorflow as tf
import numpy as np
from functions import mv
from pynput.keyboard import Key, Controller
import math
from IPython.display import clear_output
import os
keys= [Key.right,Key.down,Key.up,Key.left]
from tqdm import tqdm, trange
# try:
#     from tqdm import tqdm
# except ImportError:
#     def tqdm(x,*args, **kwargs):
#         return x

In [144]:
game=state()

In [124]:
game.getprops()

{'top': {'l0': {'starts': 534, 'ends': 631},
  'l1': {'starts': 643, 'ends': 740},
  'l2': {'starts': 753, 'ends': 850},
  'l3': {'starts': 862, 'ends': 959}},
 'left': {'l0': {'starts': 53, 'ends': 150},
  'l1': {'starts': 162, 'ends': 259},
  'l2': {'starts': 272, 'ends': 369},
  'l3': {'starts': 381, 'ends': 478}}}

In [ ]:
table=tbl(game)
keyboard = Controller()

In [153]:
table.gettbl()

,0,1,2,3
0,2.0,2.0,4.0,2.0
1,8.0,16.0,16.0,4.0
2,4.0,8.0,8.0,16.0
3,NaN,NaN,2.0,8.0


In [154]:
game.statepredict()

'going'

### Hyperparameters 

In [147]:
# num_input_neurons = len(env.reset())
# num_ouptut_neurons = env.action_space.n

num_input_neurons = 16
num_output_neurons = 4
#num_hidden_layer_neurons_1 = 256

gamma = 0.99
max_epsilon = 0.9
epsilon = max_epsilon
min_epsilon = 0.01
epsilon_decay_factor = 0.001

num_episodes = 100

learning_rate = 0.001

### training the Q-network

In [63]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Dense(128, activation='relu', input_dim = num_input_neurons),
    tf.keras.layers.Dense(128,activation='relu'),
    tf.keras.layers.Dense(num_output_neurons, activation='linear')
    
])
#opt=tf.keras.optimizers.Adam(lr=learning_rate)

model.compile(optimizer=opt,
              loss='mse',       # mean squared error
              metrics=['mae'])


# model.compile(loss='mse',
#               optimizer='Adam')

In [ ]:
model.load_weights(checkpoint_path)
loss,acc = model.evaluate(test_images, test_labels)
print("Restored model, accuracy: {:5.2f}%".format(100*acc))

In [131]:
reward_list = []

In [197]:
num_episodes=2
progress = tqdm(num_episodes )
sleep(2)
for i in range(num_episodes) :
    table.gettbl()
    tablestate = table.tablestate()
    reward_so_far = 0
    while game.statepredict()=='going':
        clear_output(wait=True)
        progress.refresh()
        action, q_vals = np.array([np.argmax(model.predict(tablestate))]), model.predict(tablestate)
        
        if np.random.rand(1) < epsilon:
            action[0] = np.random.randint(4)
            
        mv(keys[action[0]] )
        
        print(table.tbl.to_string(index=False, header=False),'\n reward=',reward, '\n', str(keys[action[0]]))

        
        sleep(0.35)
        table.gettbl()
        next_tablestate=table.tablestate()
        
        if np.array_equal(next_tablestate,tablestate):
            reward=-50
            sleep(2)
        else:
            reward = tablestate.sum()-next_tablestate.sum()
        
        if game.statepredict()=='lost':
            reward=-100

        max_q=np.max(model.predict(tablestate))
        target_q_vals = q_vals
        target_q_vals[0, action[0]] = reward + gamma * max_q
        
        model.fit(tablestate, target_q_vals, epochs=1)

        #clear_output(wait=True)
        
        

        tablestate = next_tablestate
        reward_so_far += reward
        sleep(0.1)
        

    # epsilon decay
    progress.update()
    progress.refresh()
    reward_list.append([reward_so_far, table.tbl.sum().sum()])
    game.statepredict(save=True, folder='./data/lost_games/')
    keyboard.press(Key.f5)
    sleep(0.02)
    keyboard.release(Key.f5)
    sleep(3)
    epsilon = min_epsilon + (max_epsilon - min_epsilon) * math.exp(-epsilon_decay_factor * i)






1it [01:29, 71.13s/it]

  2.0  NaN NaN  NaN
  4.0  NaN NaN  2.0
 16.0  2.0 NaN  NaN
  8.0  4.0 NaN  NaN 
 reward= -1.0 
 Key.right


KeyboardInterrupt: 

In [151]:
reward_list

[[-943.0, 312.0],
 [-1091.0, 202.0],
 [-1195.0, 268.0],
 [-2449.0, 346.0],
 [-1639.0, 324.0],
 [-688.0, 214.0],
 [0, 214.0],
 [0, 214.0],
 [0, 214.0],
 [-1050.0, 350.0],
 [-1846.0, 298.0],
 [-741.0, 184.0],
 [-996.0, 308.0],
 [-2445.0, 430.0],
 [-596.0, 228.0],
 [-1701.0, 408.0],
 [-795.0, 220.0],
 [-1742.0, 294.0],
 [-1749.0, 342.0],
 [-1342.0, 298.0],
 [-1446.0, 362.0],
 [-787.0, 144.0],
 [-639.0, 178.0],
 [-927.0, 214.0],
 [-1493.0, 216.0],
 [-1299.0, 336.0],
 [-1697.0, 322.0],
 [-944.0, 198.0],
 [-1195.0, 292.0],
 [-1291.0, 268.0],
 [-1650.0, 380.0],
 [-1490.0, 366.0],
 [-1344.0, 300.0],
 [-2547.0, 268.0],
 [-996.0, 266.0],
 [-1041.0, 214.0],
 [-1895.0, 338.0],
 [-843.0, 284.0],
 [-1795.0, 432.0],
 [-1190.0, 222.0],
 [-1086.0, 222.0],
 [-1494.0, 236.0],
 [-1343.0, 292.0],
 [-443.0, 192.0],
 [-990.0, 240.0],
 [-1092.0, 234.0],
 [-840.0, 236.0],
 [-2201.0, 468.0],
 [-1291.0, 188.0],
 [-1398.0, 330.0],
 [-1442.0, 186.0],
 [-1395.0, 294.0],
 [-945.0, 286.0],
 [-1343.0, 282.0],
 [-691.0

180.0

In [98]:
checkpoint_path = "checkpoints/"
checkpoint_dir = os.path.dirname(checkpoint_path)


# Create checkpoint callback
# cp_callback = tf.keras.callbacks.ModelCheckpoint(checkpoint_path,
#                                                  save_weights_only=True,
#                                                  verbose=1)

model.save_weights('./checkpoints/')

#### in case we have wrong gamestate

In [142]:
game.statepredict(save=True)


'going'

In [137]:
game.fit()
game.save()

/home/ainur/venv/lib/python3.6/site-packages/sklearn/linear_model/stochastic_gradient.py:183: FutureWarning: max_iter and tol parameters have been added in SGDClassifier in 0.19. If max_iter is set but tol is left unset, the default value for tol in 0.19 and 0.20 will be None (which is equivalent to -infinity, so it has no effect) but will change in 0.21 to 1e-3. Specify tol to silence this warning.
  FutureWarning)


## testing the network

In [152]:
num_episodes = 2
total_reward = []

# model = create_model()
# model.load_weights(latest)
# loss, acc = model.evaluate(test_images, test_labels)
 
sleep(2)
table.gettbl()

for _ in tqdm(range(num_episodes)):
    
    while game.statepredict()=='going':
        table.gettbl()
        tablestate = table.tablestate()
#         action = sess.run(best_action, feed_dict={input_obs: state})
        action= np.array([np.argmax(model.predict(tablestate))])
        mv(keys[action[0]])
        sleep(0.35)
        
        if np.array_equal(next_tablestate,tablestate):
            reward=-100
        else:
            reward = tablestate.sum()-next_tablestate.sum()
        
        if game.statepredict()=='lost':
            reward=-1000
            
       # state, reward, done, _ = env.step(action[0])
        total_reward.append(reward)
    keyboard.press(Key.f5)
    sleep(0.02)
    keyboard.release(Key.f5)
    sleep(3)


print("Average reward per episode: {}".format(total_reward/float(num_episodes)))





  0%|          | 0/2 [00:00<?, ?it/s]

KeyboardInterrupt: 